In [1]:
import os
import gc
import time
import math
import random
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import nvidia_smi
from tqdm.notebook import tqdm
import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader, Dataset
from torch.utils.checkpoint import checkpoint
import tokenizers
import transformers
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_cosine_schedule_with_warmup, DataCollatorWithPadding
from sklearn.model_selection import StratifiedGroupKFold

%env TOKENIZERS_PARALLELISM=true

device = "cuda" if torch.cuda.is_available() else "cpu"

env: TOKENIZERS_PARALLELISM=true


In [2]:
# Custom imports.
from utils import f2_score, seed_everything, AverageMeter, timeSince, get_vram, get_param_counts
from baseline_utils.dataset import custom_dataset, collate
from baseline_utils.model import custom_model
from baseline_utils.utils import get_max_length, get_best_threshold, get_optimizer_params

In [3]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vincenttu (use `wandb login --relogin` to force relogin)


True

In [5]:
class Args:
    def __init__(self):
        self.epochs = epochs
        self.model = "xlm-roberta-base"
        self.correlations = "../input/correlations.csv"
        self.train = "../input/train_5fold.csv"
        self.project = "LECR_0.297_baseline"
        self.project_run_root = "test"
        self.save_root = "../models/test/"
        self.fold = 0
        self.patience = 1
        self.debug = 0
        self.gradient_checkpointing = 0

In [6]:
args = Args()

epochs = args.epochs
correlations = args.correlations
train = args.train
project = args.project
project_run_root = args.project_run_root
save_root = args.save_root
os.makedirs(save_root, exist_ok=True)

fold = args.fold
patience = args.patience
gradient_checkpointing = args.gradient_checkpointing

seed = 42

class CFG:
    print_freq = 500
    num_workers = 4
    model = args.model 
    tokenizer = AutoTokenizer.from_pretrained(model)
    num_cycles = 0.5
    warmup_ratio = 0.1
    encoder_lr = 1e-5
    decoder_lr = 1e-4
    eps = 1e-6
    betas = (0.9, 0.999)
    batch_size = 32
    weight_decay = 0.01
    max_grad_norm = 0.012
    max_len = 512

In [7]:
# =========================================================================================
# Train function loop
# =========================================================================================
def train_fn(train_loader, model, criterion, optimizer, epoch, scheduler, device, cfg):
    model.train()
    scaler = torch.cuda.amp.GradScaler(enabled = True)
    losses = AverageMeter()
    start = end = time.time()
    global_step = 0
    for step, (inputs, target) in enumerate(train_loader):
        inputs = collate(inputs)
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        target = target.to(device)
        batch_size = target.size(0)
        with torch.cuda.amp.autocast(enabled = True):
            y_preds = model(inputs)
            loss = criterion(y_preds.view(-1), target)
        losses.update(loss.item(), batch_size)
        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), cfg.max_grad_norm)
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()
        global_step += 1
        scheduler.step()
        end = time.time()
        if step % cfg.print_freq == 0 or step == (len(train_loader) - 1):
            print('Epoch: [{0}][{1}/{2}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  'Grad: {grad_norm:.4f}  '
                  'LR: {lr:.8f}  '
                  .format(epoch + 1, 
                          step, 
                          len(train_loader), 
                          remain = timeSince(start, float(step + 1) / len(train_loader)),
                          loss = losses,
                          grad_norm = grad_norm,
                          lr = scheduler.get_lr()[0]))
            if args.debug:
                get_vram()
        if not args.debug and step % (cfg.print_freq * 6) == 0:
            get_vram()

    return losses.avg

# =========================================================================================
# Valid function loop
# =========================================================================================
def valid_fn(valid_loader, model, criterion, device, cfg):
    losses = AverageMeter()
    model.eval()
    preds = []
    start = end = time.time()
    for step, (inputs, target) in enumerate(valid_loader):
        inputs = collate(inputs)
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        target = target.to(device)
        batch_size = target.size(0)
        with torch.no_grad():
            y_preds = model(inputs)
        loss = criterion(y_preds.view(-1), target)
        losses.update(loss.item(), batch_size)
        preds.append(y_preds.sigmoid().squeeze().to('cpu').numpy().reshape(-1))
        end = time.time()
        if step % cfg.print_freq == 0 or step == (len(valid_loader) - 1):
            print('EVAL: [{0}/{1}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  .format(step, 
                          len(valid_loader),
                          loss = losses,
                          remain = timeSince(start, float(step + 1) / len(valid_loader))))
            
        if step % (cfg.print_freq * 6) == 0:
            get_vram()
            
    predictions = np.concatenate(preds, axis = 0)
    
    return losses.avg, predictions

In [8]:
# Seed everything
seed_everything(seed)

cfg = CFG()

# Read data
correlations = pd.read_csv("../input/correlations.csv")
train = pd.read_csv("../input/train_5fold.csv")

# Get max length
get_max_length(train, CFG)

100%|██████████| 615170/615170 [00:44<00:00, 13848.11it/s]

max_len: 172


In [9]:
# Split train & validation.
x_train = train[train['fold'] != fold]
x_val = train[train['fold'] == fold]
valid_labels = x_val['target'].values
train_dataset = custom_dataset(x_train, cfg)
valid_dataset = custom_dataset(x_val, cfg)
train_loader = DataLoader(
    train_dataset, 
    batch_size = cfg.batch_size, 
    shuffle = True, 
    num_workers = cfg.num_workers, 
    pin_memory = True, 
    drop_last = True
)
valid_loader = DataLoader(
    valid_dataset, 
    batch_size = cfg.batch_size, 
    shuffle = False, 
    num_workers = cfg.num_workers, 
    pin_memory = True, 
    drop_last = False
)

In [10]:
# Model.
model = custom_model(cfg, gradient_checkpointing=gradient_checkpointing)
_ = model.to(device)

# Optimizer.
optimizer_parameters = get_optimizer_params(
    model, 
    encoder_lr = cfg.encoder_lr, 
    decoder_lr = cfg.decoder_lr,
    weight_decay = cfg.weight_decay
)
optimizer = AdamW(
    optimizer_parameters, 
    lr = cfg.encoder_lr, 
    eps = cfg.eps, 
    betas = cfg.betas
)

# Scheduler.
num_train_steps = int(len(x_train) / cfg.batch_size * epochs)
num_warmup_steps = num_train_steps * cfg.warmup_ratio
scheduler = get_cosine_schedule_with_warmup(
    optimizer, 
    num_warmup_steps = num_warmup_steps, 
    num_training_steps = num_train_steps, 
    num_cycles = cfg.num_cycles
    )

# Criterion.
criterion = nn.BCEWithLogitsLoss(reduction = "mean")

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
print("Printing GPU stats...")
get_vram()

Printing GPU stats...
Device 0: b'NVIDIA A40', Memory : (74.83% free): 13.496484911069274 (total), 10.100000536069274 (free), 3.396484375 (used)


In [13]:
# Project configuration.
print("Printing GPU stats...")
get_vram()

cfg_params = [i for i in dir(cfg) if "__" not in i]
cfg_params = dict(zip(cfg_params, [getattr(cfg, i) for i in cfg_params]))
total_params, trainable_params, nontrainable_params = get_param_counts(model)
cfg_params.update({
    "total_params": total_params,
    "trainable_params": trainable_params,
    "nontrainable_params": nontrainable_params
})

# Initialize run.
run = wandb.init(project=project, config=cfg_params, name=f"{project_run_root}_fold{fold}", dir="/tmp")

Printing GPU stats...
Device 0: b'NVIDIA A40', Memory : (74.83% free): 13.496484911069274 (total), 10.100000536069274 (free), 3.396484375 (used)


wandb: wandb version 0.13.7 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [14]:
# Training & validation loop.
best_score, cnt = 0, 0
for epoch in range(epochs):
    start_time = time.time()

    # Train.
    avg_loss = train_fn(train_loader, model, criterion, optimizer, epoch, scheduler, device, cfg)

    # Validation.
    avg_val_loss, predictions = valid_fn(valid_loader, model, criterion, device, cfg)

    # Compute f2_score.
    score, threshold = get_best_threshold(x_val, predictions, correlations)

    # Logging.
    elapsed = time.time() - start_time
    print(f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
    print(f'Epoch {epoch+1} - Score: {score:.4f} - Threshold: {threshold:.5f}')

    run.log({
        "epoch": epoch,
        "avg_train_loss": avg_loss,
        "avg_val_loss": avg_val_loss,
        "f2_score": score,
        "threshold": threshold
    })

    # Saving & early stopping.
    if score > best_score:
        best_score = score
        print(f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')
        save_m = f"{cfg.model.replace('/', '-')}_fold{fold}_ep{epoch}.pth"
        save_p = os.path.join(save_root, save_m)
        torch.save(model.state_dict(), save_p)

        # W&B save model as artifact.
        artifact = wandb.Artifact(cfg.model.replace('/', '-'), type='model')
        artifact.add_file(save_p, name=f"fold{fold}_ep{epoch}.pth")
        run.log_artifact(artifact)

        val_predictions = predictions
    elif patience != -1 and patience > 0:
        cnt += 1
        if cnt == patience:
            print(f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')
            save_p = os.path.join(save_root, f"{cfg.model.replace('/', '-')}_fold{fold}_ep{epoch}.pth")
            torch.save(model.state_dict(), save_p)

            # W&B save model as artifact.
            artifact = wandb.Artifact(cfg.model.replace('/', '-'), type='model')
            artifact.add_file(save_p, name=f"fold{fold}_ep{epoch}.pth")
            run.log_artifact(artifact)

            val_predictions = predictions
            break

torch.cuda.empty_cache()
gc.collect()

# Get best threshold.
best_score, best_threshold = get_best_threshold(x_val, val_predictions, correlations)
print(f'Our CV score is {best_score} using a threshold of {best_threshold}')

run.finish()

Epoch: [1][0/15381] Elapsed 0m 0s (remain 74m 9s) Loss: 0.8532(0.8532) Grad: inf  LR: 0.00000000  
Device 0: b'NVIDIA A40', Memory : (61.40% free): 13.496484911069274 (total), 8.287500536069274 (free), 5.208984375 (used)
Epoch: [1][500/15381] Elapsed 0m 48s (remain 24m 4s) Loss: 0.5878(0.7907) Grad: 6.3956  LR: 0.00000065  
Device 0: b'NVIDIA A40', Memory : (37.69% free): 13.496484911069274 (total), 5.086328661069274 (free), 8.41015625 (used)
Epoch: [1][1000/15381] Elapsed 1m 37s (remain 23m 25s) Loss: 0.2023(0.5462) Grad: 4.6545  LR: 0.00000130  
Device 0: b'NVIDIA A40', Memory : (37.22% free): 13.496484911069274 (total), 5.023828661069274 (free), 8.47265625 (used)
Epoch: [1][1500/15381] Elapsed 2m 26s (remain 22m 38s) Loss: 0.1050(0.4402) Grad: 9.4569  LR: 0.00000195  
Device 0: b'NVIDIA A40', Memory : (37.08% free): 13.496484911069274 (total), 5.004297411069274 (free), 8.4921875 (used)
Epoch: [1][2000/15381] Elapsed 3m 15s (remain 21m 47s) Loss: 0.2976(0.3828) Grad: 7.6789  LR: 0.00

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /tmp/ipykernel_1882/978766885.py:7 in <module>                                            │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_1882/978766885.py'                   │
│ /tmp/ipykernel_1882/106053499.py:23 in train_fn                                           │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_1882/106053499.py'                   │
│                                                                                           │
│ /opt/conda/envs/python37/lib/python3.7/site-packages/torch/cuda/amp/grad_scaler.py:338 in │
│ step                                                                                      │
│                                                                                           │
│   335 │   │                                                                               │
│   336 │   │   assert len(optimizer_state["found_inf_per_device"]) > 0, "No inf checks wer │
│   337 │   │                                                                               │
│ ❱ 338 │   │   retval = self._maybe_opt_step(optimizer, optimizer_state, *args, **kwargs)  │
│   339 │   │                                                                               │
│   340 │   │   optimizer_state["stage"] = OptState.STEPPED                                 │
│   341                                                                                     │
│                                                                                           │
│ /opt/conda/envs/python37/lib/python3.7/site-packages/torch/cuda/amp/grad_scaler.py:284 in │
│ _maybe_opt_step                                                                           │
│                                                                                           │
│   281 │                                                                                   │
│   282 │   def _maybe_opt_step(self, optimizer, optimizer_state, *args, **kwargs):         │
│   283 │   │   retval = None                                                               │
│ ❱ 284 │   │   if not sum(v.item() for v in optimizer_state["found_inf_per_device"].values │
│   285 │   │   │   retval = optimizer.step(*args, **kwargs)                                │
│   286 │   │   return retval                                                               │
│   287                                                                                     │
│                                                                                           │
│ /opt/conda/envs/python37/lib/python3.7/site-packages/torch/cuda/amp/grad_scaler.py:284 in │
│ <genexpr>                                                                                 │
│                                                                                           │
│   281 │                                                                                   │
│   282 │   def _maybe_opt_step(self, optimizer, optimizer_state, *args, **kwargs):         │
│   283 │   │   retval = None                                                               │
│ ❱ 284 │   │   if not sum(v.item() for v in optimizer_state["found_inf_per_device"].values │
│   285 │   │   │   retval = optimizer.step(*args, **kwargs)                                │
│   286 │   │   return retval                                                               │
│   287                                                                                     │
╰───────────────────────────────────────────────────────────────────────────────────────────╯
KeyboardInterrupt

In [15]:
run.finish()